In [ ]:
# ==========================================
# 🛠️ SCRIPT 1: SETUP & INSTALLATION
# ==========================================
import os
import sys

# 1. Clone the Repository
if not os.path.exists('Personal-Voice-Clone'):
    print("📂 Cloning Repository...")
    !git clone https://github.com/micccon/Personal-AI-Clone.git
    %cd Personal-Voice-Clone
else:
    %cd Personal-Voice-Clone
    print("📂 Repo already cloned.")

# 2. Install System Audio Tools
print("⚙️ Installing System Libraries...")
!apt-get update -y > /dev/null 2>&1
!apt-get install -y ffmpeg > /dev/null 2>&1

# 3. Install Numpy Constraint
print("🛡️ Installing Numpy Constraint...")
!pip install "numpy<2.0.0" --force-reinstall

# 4. Install Llama-3 (GPU)
print("🚀 Installing Llama-3 (GPU)...")
!pip install llama-cpp-python \
  --prefer-binary \
  --extra-index-url=https://abetlen.github.io/llama-cpp-python/whl/cu121 \
  --no-cache-dir

# 5. Install App Dependencies (Voice is installed but optional in usage)
print("⚙️ Installing App Dependencies...")
!pip install -q \
    elevenlabs \
    streamlit \
    python-dotenv \
    "transformers==4.46.1"

print("\n" + "="*50)
print("✅ INSTALL COMPLETE.")
print("👉 Please click 'Restart Session' (top menu) if prompted.")
print("="*50)

In [ ]:
# ==========================================
# 🔑 SCRIPT 2: CONFIGURATION
# ==========================================
import os
from getpass import getpass

print("📝 SETTING UP...\n")

# 1. Ngrok (REQUIRED for TCP Tunnel)
print("1️⃣ Enter Ngrok Auth Token (Verified account required for TCP):")
print("   (Get it from https://dashboard.ngrok.com/get-started/your-authtoken)")
ngrok_token = getpass()

# 2. ElevenLabs (Optional)
print("\n2️⃣ Enter ElevenLabs API Key (Optional - press Enter to skip):")
eleven_key = getpass()
print("\n3️⃣ Enter Voice ID (Optional - press Enter to skip):")
voice_id = input()

# 3. Model Selection (Custom vs Generic)
print("\n4️⃣ Model Configuration")
print("   Press ENTER to use the generic Llama-3 demo model.")
print("   OR paste a direct download link to your own .gguf model (HuggingFace/Dropbox).")
custom_model_url = input("   Model URL: ").strip()

# --- SAVE CONFIGURATION ---
if ngrok_token.strip(): 
    os.environ["NGROK_AUTH_TOKEN"] = ngrok_token

with open(".env", "w") as f:
    if eleven_key.strip(): f.write(f"ELEVENLABS_API_KEY={eleven_key}\n")
    if voice_id.strip(): f.write(f"VOICE_ID={voice_id}\n")

# --- DOWNLOAD MODEL ---
if not os.path.exists("models"): os.makedirs("models")
model_path = "models/inference_model.gguf"

if custom_model_url:
    print(f"\n⬇️ Downloading custom model from provided URL...")
    !wget -q "{custom_model_url}" -O {model_path}
else:
    # Default to a high-quality quantized Llama-3-8B-Instruct
    if not os.path.exists(model_path):
        print("\n⬇️ Downloading generic Llama-3 model for demo purposes...")
        !wget -q https://huggingface.co/bartowski/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf -O {model_path}

with open(".env", "a") as f: f.write(f"MODEL_PATH={model_path}\n")
print("✅ Configuration Saved.")

In [ ]:
# ==========================================
# 🚀 SCRIPT 3: LAUNCH APP
# ==========================================
from pyngrok import ngrok, conf
import os

# 1. Authenticate
if "NGROK_AUTH_TOKEN" in os.environ:
    ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])
else:
    print("❌ Error: Ngrok token missing. Run Cell 2.")

# 2. Kill old tunnels
ngrok.kill()

# 3. Open TCP Tunnel
# We use TCP because standard HTTP tunnels often fail with Streamlit WebSockets on Colab.
try:
    tunnel = ngrok.connect(8501, "tcp")
    
    # Convert "tcp://0.tcp.ngrok.io:12345" -> "http://0.tcp.ngrok.io:12345"
    public_url = tunnel.public_url.replace("tcp://", "http://")
    
    print("\n" + "="*60)
    print(f"🚀 YOUR APP IS LIVE! CLICK HERE: {public_url}")
    print("="*60 + "\n")
    
    # 4. Run Streamlit
    !streamlit run app.py \
      --server.port 8501 \
      --server.address 0.0.0.0 \
      --server.headless true
      
except Exception as e:
    print(f"❌ Tunnel Error: {e}")
    print("⚠️ Note: TCP tunnels require a verified Ngrok account. If you haven't added a payment method to Ngrok, this will fail.")